In [2]:
import tensorflow as tf
import os
import pickle
import numpy as np

CIFAR_DIR = "../cifar-10-batches-py"
print(os.listdir(CIFAR_DIR))

['data_batch_1', 'readme.html', 'batches.meta', 'data_batch_2', 'data_batch_5', 'test_batch', 'data_batch_4', 'data_batch_3', '.ipynb_checkpoints']


In [12]:
def load_data(filename):
    """ read data from data file."""
    with open(filename, 'rb') as f:
        data = pickle.load(f,encoding='iso-8859-1')
        return data['data'], data['labels']

class CifarData:
    # 初始化
    def __init__(self, filenames, need_shuffle):
        all_data = [] # 存放数据
        all_labels = [] # 存放标签
        # 5个file 所以需要循环
        for filename in filenames:
            data, labels = load_data(filename)
            for item, label in zip(data, labels):
                if label in [0,1]:
                    all_data.append(item)
                    all_labels.append(label)
        self._data = np.vstack(all_data)
        self._data = self._data / 127.5 -1
        self._labels = np.hstack(all_labels)
        print(self._data.shape)
        print(self._labels.shape)
        
        self._num_examples = self._data.shape[0]
        self._need_shuffle = need_shuffle
        self._indicator = 0
        if self._need_shuffle:
            self._shuffle_data()
    
    # 打乱数据
    def _shuffle_data(self):
        # [0,1,2,3,4] --> [4,2,0,1,3]
        p = np.random.permutation(self._num_examples)
        self._data = self._data[p]
    
    def next_batch(self, batch_size):
        """return batch_size examples as a batch."""
        end_indicator = self._indicator + batch_size
        if end_indicator > self._num_examples:
            if self._need_shuffle:
                self._shuffle_data()
                self._indicator = 0
                end_indicator = batch_size
            else:
                raise Exception('have no more examples')
        if end_indicator > self._num_examples:
            raise Exception("batch size larger than all examples")
        batch_data = self._data[self._indicator: end_indicator]
        batch_labels = self._labels[self._indicator: end_indicator]
        return batch_data, batch_labels
    
train_filenames = [os.path.join(CIFAR_DIR, 'data_batch_%d' % i) for i in range(1,6)]
test_filenames = [os.path.join(CIFAR_DIR, 'test_batch')]

train_data = CifarData(train_filenames, True)
test_data = CifarData(test_filenames, False)

# batch_data, batch_labels = train_data.next_batch(10)
# print(batch_data,'\n', batch_labels)

(10000, 3072)
(10000,)
(2000, 3072)
(2000,)
[[ 0.84313725  0.8745098   0.86666667 ...  0.2627451   0.25490196
   0.24705882]
 [ 0.99215686  0.99215686  0.99215686 ...  1.          1.
   1.        ]
 [ 0.23137255  0.15294118  0.27843137 ... -0.09803922 -0.1372549
  -0.12941176]
 ...
 [ 0.98431373  1.          1.         ... -0.27058824 -0.28627451
  -0.30980392]
 [-0.23137255 -0.27843137 -0.41960784 ... -0.36470588 -0.35686275
  -0.36470588]
 [-0.37254902 -0.34117647 -0.29411765 ... -0.70980392 -0.7254902
  -0.70980392]] 
 [1 1 0 0 1 0 1 1 1 0]


In [4]:
x = tf.placeholder(tf.float32, [None, 3072])
# [None]
y = tf.placeholder(tf.int64, [None])

# (3072, 1)
w = tf.get_variable('w', [x.get_shape()[-1], 1],
                   initializer=tf.random_normal_initializer(0, 1))
# (1, )
b = tf.get_variable('b', [1],
                   initializer=tf.constant_initializer(0.0))

# [None, 3072] * [3072, 1] = [None, 1]
y_ = tf.matmul(x, w) + b

# [None, 1]
p_y_1 = tf.nn.sigmoid(y_)
# [None, 1]
y_reshaped = tf.reshape(y, (-1, 1))
y_reshaped_float = tf.cast(y_reshaped, tf.float32)
loss = tf.reduce_mean(tf.square(y_reshaped_float - p_y_1))

# bool
predict = p_y_1 > 0.5
# [1,0,1,1,1,0,0,0]
correct_prediction = tf.equal(tf.cast(predict, tf.int64), y_reshaped)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float64))

with tf.name_scope('train_op'):
    train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)

Instructions for updating:
Colocations handled automatically by placer.


In [14]:
# 一些预定义的变量值
init = tf.global_variables_initializer()

batch_size = 20
train_steps = 1000
test_steps = 1000

with tf.Session() as sess:
    sess.run(init)
    for i in range(train_steps):
        batch_data, batch_labels = train_data.next_batch(batch_size)
        loss_val, acc_val, _ = sess.run(
            [loss, accuracy, train_op],
            feed_dict = {
                x: batch_data,
                y: batch_labels})
        if i% 500 == 0:
            print('[Train] step: %d, loss: %4.5f, acc: %4.5f' %(i+1, loss_val, acc_val))

[Train] step: 1, loss: 0.42855, acc: 0.55000
[Train] step: 501, loss: 0.25001, acc: 0.75000
